Merhaba!

Bu çalışma sayfasında verilen çeşitli müşteri bilgilerine dayanarak müşterilerin kasko sigortası alıp almayacağını tahmin etmeye çalışacağım.

Fikir ve yorumlarınız benim için değerlidir.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("/kaggle/input/carinsurance/carInsurance_train.csv")
train.head()

Elimizdeki data da Id numaraları dahil 20 değişken var bu değişkenler yaş, medeni durum, eğitim durumu, görüşme başlama ve bitiş zamanları gibi detaylı bilgiler dataset'in açıklamalarında mevcut. Elimizdeki değişkenleri görmek ve bu değişkenler hakkında basit bilgilere sahip olmak için;

In [ ]:
train.columns

In [ ]:
train.describe()

In [ ]:
train.isnull().sum() / len(train)

Veride en fazla boş satırlar Outcome değişkeninde var model kurmadan önce veriyi daha iyi hale getirmek için aşağıdaki adımları uygulayacağım.

 1- Önce ayların bulunduğu LastContactMonth değişkenini aylıklardan dönemlere çevireceğim bir şekilde mevsimsellik var mı yok mu hangi mevsimlerde ortalama satış artmış veya azalmış görmek için.
 
 2- Konuşma başlama ve bitiş sürelerini birbirlerinden çıkarıp bir skala haline getireceğim. Konuşma süresinin kasko sigortasına etkisi var mı yok mu konuşma uzadıkça hedef değişkenim ne oluyor görmek için. Bunu yaptıktan sonra bu iki değişkeni datadan atacağım.
 
 3- Yaşları aynı şekilde bir skalaya getirip genç, orta yaşlı, yaşlı vs gibi gruplayacağım.
 
 4- Outcome değişkenini datadan çıkaracağım çünkü %76'sı boş, Job değişkeni boş olanları silebilirim çünkü az miktarda bir veri fakat education için hem yaş hemde meslek bilgilerini kullanarak kişilerin eğitim durumu tahmin etmem daha iyi olur.
 
 5- Tüm değişkenleri encode edip veriyi bağımlı bağımsız değişken olarak bölüp modellemeye hazır hale getireceğim.
 
 Bu işlmelere geçmeden önce data hakkında daha fazla bilgiye sahip olmak için pairplot grafiğine bakılabilir.
 
 

In [ ]:
import seaborn as sns

sns.pairplot(data = train, hue = "CarInsurance")

**AYLAR**

In [ ]:
yeni_degiskenler = []

i = 0

while i < 4001:
    
    if train["LastContactMonth"][i] == "jan":
        yeni_degiskenler.append(1)
    elif train["LastContactMonth"][i] == "feb":
        yeni_degiskenler.append(1)
    elif train["LastContactMonth"][i] == "mar":
        yeni_degiskenler.append(1)
    elif train["LastContactMonth"][i] == "apr":
        yeni_degiskenler.append(2)
    elif train["LastContactMonth"][i] == "may":
        yeni_degiskenler.append(2)
    elif train["LastContactMonth"][i] == "jun":
        yeni_degiskenler.append(2)
    elif train["LastContactMonth"][i] == "jul":
        yeni_degiskenler.append(3)
    elif train["LastContactMonth"][i] == "aug":
        yeni_degiskenler.append(3)
    elif train["LastContactMonth"][i] == "sep":
        yeni_degiskenler.append(3)
    elif train["LastContactMonth"][i] == "oct":
        yeni_degiskenler.append(4)
    elif train["LastContactMonth"][i] == "nov":
        yeni_degiskenler.append(4)
    elif train["LastContactMonth"][i] == "dec":
        yeni_degiskenler.append(4)
        
    i += 1

In [ ]:
len(yeni_degiskenler)

Ayları dönemlere çevirdikten sonra konuşma sürelerini hesaplamak için:

In [ ]:
konusma = pd.to_datetime(train["CallEnd"]) - pd.to_datetime(train["CallStart"])

konusma = pd.to_numeric(konusma) / 1000000000
konusma = pd.DataFrame(konusma)

a = pd.concat([train.CarInsurance, konusma], names = ("sonuc", "saniye"), axis = 1)

sns.pairplot(data = a, hue = "CarInsurance")

konusma.loc[(konusma[0] >= 0) & (konusma[0] <= 126), "dakika"] = 1
konusma.loc[(konusma[0] > 126) & (konusma[0] <= 232), "dakika"] = 2
konusma.loc[(konusma[0] > 232) & (konusma[0] <= 460), "dakika"] = 3
konusma.loc[(konusma[0] > 460) & (konusma[0] <= 3253), "dakika"] = 4

Aslında burası bize önemli bir şey söylüyor. Alttaki grafikte görülen şey eğer konuşma süresi kısa ise hedef değişken cevabı 0 olması daha muhtemel tersi yani konuşma süresi uzun ise cevabın 1 olması daha muhtemel gibi.

In [ ]:
dakika = pd.DataFrame(konusma["dakika"])
yeni_degiskenler = pd.DataFrame(yeni_degiskenler, columns = ["quarter"])

yeni_degiskenler = pd.concat([yeni_degiskenler, dakika], axis = 1)
yeni_degiskenler.head()

Şimdi de yaşları skala edelim.

In [ ]:
i = 0
yas = []

while i < 4000:
    
    if train["Age"][i] > 0 and train["Age"][i] < 26:
        yas.append(1)
    elif train["Age"][i] >=26 and train["Age"][i] < 35:
        yas.append(2)
    elif train["Age"][i] >= 35 and train["Age"][i] < 45:
        yas.append(3)
    elif train["Age"][i] >= 45:
        yas.append(4)
        
    i += 1

yas = pd.DataFrame(yas, columns = ["yas"])
yeni_degiskenler = pd.concat([yeni_degiskenler, yas], axis = 1)
yeni_degiskenler.head()

Çıkarılan bu 3 yeni değişkeni train'in içine koyma

In [ ]:
train = pd.concat([yeni_degiskenler, train], axis = 1)
train.head()

Education içindeki boş verileri doldurmak için yağtığım şey şu benim elimde kişilerin meslekleri ve yaşları var eğer aynı mesleğe sahip insaları datadan çekersem bunların ağırlıklı olarak hangi eğitim düzeyinde oldukları ve eğitim düzeyinin yaşlara dağılımı gibi önemli bilgileri el edebilirim örneğin işi manager olanların büyük çoğunluğu tertiary seviyesinde eğitime sahip olduklarını varsayılım (ki öyle) fakat bir yandan da yaşı 25-30 olup eğitimi tertiary bile olsa mesleği manager olamayacak (çünkü henüz gençler) insanlar olacaktır veya aynı şekilde yaşı 65 olanlar genellikle tertiary eğitim durumu olsa bile retired olarak karşımıza çıkacaklar bunun için education sütünundaki boşlukları hem meslek hemde yaş bilgileri ile beraber doldurmanın daha doğru olacağına inanıyorum.

In [ ]:
management = train.loc[train["Job"] == "management" ]
blue_collar = train.loc[train["Job"] == "blue-collar"]
technician = train.loc[train["Job"] == "technician"]
admin = train.loc[train["Job"] == "admin."]
services = train.loc[train["Job"] == "services"]
retired = train.loc[train["Job"] == "retired"]
self_employed = train.loc[train["Job"] == "self-employed"]
student = train.loc[train["Job"] == "student"]

management["Education"].value_counts() # tertiary 751 / secondary 94 / primary 22     
blue_collar["Education"].value_counts() # secondary 430 / primary 281 / tertiary 17
technician["Education"].value_counts() # secondary 446 / tertiary 177 / primary 16
admin["Education"].value_counts() # secondary 374 / tertiary 62 / primary 9            
services["Education"].value_counts() # secondary 269 / primary 27 / tertiary 25      
retired["Education"].value_counts() # secondary 100 / primary 93 / tertiary 37
self_employed["Education"].value_counts() # tertiary 84 / secondary 43 / primary 6
student["Education"].value_counts() # secondary 67 / tertiary 31 / primary 8

management["Age"].mean() #40
blue_collar["Age"].mean() #40
technician["Age"].mean() #39
admin["Age"].mean() # 39
services["Age"].mean() # 38
retired["Age"].mean() #63
self_employed["Age"].mean() # 41
student["Age"].mean() #25

train["Education"] = train["Education"].replace(np.nan, "?")


i = 0

while i < 4000:
    if train["Age"][i] >= 35 and train["Job"][i] == "management" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "tertiary")
    elif train["Age"][i] < 35 and train["Job"][i] == "management" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "secondary")
    i += 1

i = 0

while i < 4000:
    if train["Job"][i] == "admin." and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "secondary")
    
    i += 1

i = 0

while i < 4000:
    if train["Job"][i] == "services" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "secondary")
    
    i += 1

i = 0

while i < 4000:
    if train["Job"][i] == "services" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "secondary")
    
    i += 1

i = 0

while i < 4000:
    if train["Age"][i] >= 28 and train["Job"][i] == "student" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "tertiary")
    elif train["Age"][i] < 28 and train["Job"][i] == "student" and train["Education"][i] == "?":
        train["Education"][i] = train["Education"][i].replace("?", "secondary")
    i += 1

train["Education"] = train["Education"].replace("?", "secondary")

In [ ]:
train.isnull().sum()

Communacation değişkenindeki boşları doldurmak için daha basit bir yol izleyeceğim çünkü baktığım zaman şöyle bir sonuç çıkıyor.

Genel olarak telefon kullanılmıyor hücresel veri daha fazla ve telefonu yaşlılar daha fazla kullanıyor fakat bu kullanım oranı yaşlılarda bile az yaşlıların %60'ından fazlası cellular kullanıyor. Bu yüzden mantıklı bir ayrım bulamadığım için tüm boşluklara cellular yerleştirdim ve job kısmındaki 19 boş satırı ve outcome, ıd gibi geresiz değişkenleri datadan çıkardım.

In [ ]:
train["Communication"] = train["Communication"].fillna("cellular")
train.isnull().sum()

In [ ]:
train = train.drop("Outcome", axis = 1)
train = train.drop("CallEnd", axis = 1)
train = train.drop("CallStart", axis = 1)
train = train.drop("Id", axis = 1)

train = train.dropna()
train.isnull().sum()

Artık boş veri kalmadı yeni değişkenler bulup bunları dataya ekledik şimdi model kurmak için hazırlıkları yapabilirim.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

cols = ["Job", "Marital", "Education", "Communication", "LastContactMonth"]

for i in cols:
    train[i] = le.fit_transform(train[i])


x = train.iloc[:, 0:17]
y = train.iloc[:, 17:18]


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)


**MODELLER**

RANDOM FOREST

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_tahmin = rf.predict(x_test)

rf_skor = accuracy_score(y_test, rf_tahmin)
rf_skor

XGBOOOST

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(x_train, y_train)
xgb_tahmin = xgb.predict(x_test)
accuracy_score(y_test, xgb_tahmin)

LIGHTGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()
lgb.fit(x_train, y_train)
lgb_tahmin = lgb.predict(x_test)
accuracy_score(y_test, lgb_tahmin)

Eğer modele yeni değişkenler eklemeyip sadece olduğu gibi bir model kursaydık %66 - %70 arası bir accuracy elde etmiş olacaktık ama yeni değişkenler ve boş satırları doldurma işlmeleri ile tune etmeden dahai %83 gibi skorlar elde edebiliyoruz.

Model başarısını daha da arttıkmak için:

Tune işlemi denenebilir.
Yeni değişkenlerin skalaları değiştirilerek denenebilir.
Feature Selection işlemi yapılabilir. Örnek için --> [https://www.kaggle.com/alpertemel/mobile-price-classification-atemel](http://)

Okuduğunuz için teşekkürler.
Tekrar görüşmek üzere :))